In [10]:
import pyarrow.parquet as pq
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm

In [3]:
DATA_PATH = '/cmnfs/data/proteomics/Prosit_unmod/intensity/no_aug'

CID = 1
HCD = 2

In [ ]:
for ds_type in ['train', 'val', 'test']:
    file_path = DATA_PATH + '_' + ds_type + '.parquet'
    file = pq.ParquetFile(file_path)
    print(f'Start processing dataset: {os.path.basename(file_path)}')

    column_names = next(file.iter_batches(batch_size=1)).to_pandas().columns
    all_cid = pd.DataFrame(columns=column_names)
    all_hcd = pd.DataFrame(columns=column_names)
    # iterate over batches of the file
    total_seqs = 0
    for batch in tqdm(file.iter_batches()):
        total_seqs += len(batch)
        df = batch.to_pandas()
        all_cid = pd.concat([df[df['method_nbr'] == 1], all_cid])
        all_hcd = pd.concat([df[df['method_nbr'] == 2], all_hcd])
    
    all_cid.to_parquet('/cmnfs/proj/bmpc_dlomix/datasets/parquet/no_aug' + '_cid_' + ds_type + '.parquet')
    all_hcd.to_parquet('/cmnfs/proj/bmpc_dlomix/datasets/parquet/no_aug' + '_hcd_' + ds_type + '.parquet')